In [48]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import pandas as pd
import os
import cv2

## Define parameters

In [49]:
learning_rate = 1e-3
input_size = 300
output_size = 100
batch_size = 10
num_epochs = 500
num_workers = 0
train_dir = os.path.join(os.getcwd(), 'data', 'train')
test_dir = os.path.join(os.getcwd(), 'data', 'test')
dtype_to_use = torch.float32

## Define the neural network architecture
Layer options (More info at https://pytorch.org/docs/stable/nn.html):
+ Linear: fully connected layer
+ Conv1d/Conv2d: Convolutional layers
+ BatchNorm2d/LayerNorm/InstanceNorm2d: Normalization layers
+ Dropout: Dropout layer
+ MaxPool2d/AvgPool2d: Pooling layers

In [50]:
class NN(pl.LightningModule):
    def __init__(self, learning_rate, input_size, output_size):
        super(NN, self).__init__()
        self.learning_rate = learning_rate
        self.layer1 = nn.Conv2d(1, 32, kernel_size=4, stride=1, padding="same", dtype=dtype_to_use)
        self.layer2 = nn.Conv2d(32, 64, kernel_size=4, stride=1, padding="same", dtype=dtype_to_use)
        self.layer4 = nn.Linear(64*33**2, 200, dtype= dtype_to_use)
        self.layer5 = nn.Linear(200, 200, dtype=dtype_to_use)
        #self.layer6 = nn.Linear(200, output_size*2, dtype= dtype_to_use)

        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)

        self.relu = nn.ReLU()
        #self.loss_fn = nn.MSELoss()

        self.training_step_outputs = []
        self.testing_step_outputs = []
    
    def forward(self, x):
        current_batch_size = x.shape[0]
        x = x.view(current_batch_size, 1, input_size, input_size)
        x = self.pool(self.relu(self.layer1(x)))
        x = self.pool(self.relu(self.layer2(x)))
        #x = self.pool(self.relu(self.layer3(x)))
        x = x.view(current_batch_size, -1)
        x = self.layer4(x)
        x = self.layer5(x)
        #x = self.layer6(x)
        x = x.view(current_batch_size, -1, output_size)
        return x
    
    def loss_fn(self, outputs, targets):
        abs_diff = torch.abs(outputs - targets)
        avg_distance = torch.mean(abs_diff)
        return avg_distance

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)
    
    def on_train_epoch_end(self):
        if self.current_epoch == self.trainer.max_epochs - 1:
            # Calculate distances for the last epoch
            avg_distances, max_distances = self.calculate_distances(self.training_step_outputs)
            for i, avg_distance in enumerate(avg_distances):
                self.logger.experiment.add_scalar("train/avg_distance", avg_distance, i)
                self.logger.experiment.add_scalar("train/max_distance", max_distances[i], i)	

    def on_test_end(self):
        if self.current_epoch >= self.trainer.max_epochs - 1:
            # Calculate distances for the last epoch
            avg_distances, max_distances = self.calculate_distances(self.testing_step_outputs)
            for i, avg_distance in enumerate(avg_distances):
                self.logger.experiment.add_scalar("test/avg_distance", avg_distance, i)
                self.logger.experiment.add_scalar("test/max_distance", max_distances[i], i)

            

    def calculate_distances(self, output_target_pairs):
        avg_distances, max_distances = [], []
        for output, target in output_target_pairs:
            avg_distance, max_distance = self.calculate_distance(output, target)
            avg_distances.append(avg_distance)
            max_distances.append(max_distance)

        return avg_distances, max_distances

    def calculate_distance(self, output, target):
        abs_diff = torch.abs(output - target)
        avg_distance = torch.mean(abs_diff)
        max_distance = torch.max(abs_diff)
        return (avg_distance.item(), max_distance.item())
    

    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self.forward(inputs)
        loss = self.loss_fn(outputs, targets)
        if torch.isnan(loss):
            self.trainer.should_stop = True
        self.log("train_loss", loss)
        if self.current_epoch == self.trainer.max_epochs - 1:
            for output, target in zip(outputs, targets):
                self.training_step_outputs.append((output, target))
        else:
            self.training_step_outputs = []
        return loss
    
    def test_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self.forward(inputs)
        loss = self.loss_fn(outputs, targets)
        self.log("test_loss", loss)
        if self.current_epoch >= self.trainer.max_epochs - 1:
            for output, target in zip(outputs, targets):
                self.testing_step_outputs.append((output, target))
        else:
            self.testing_step_outputs = []
        return loss


## Custom dataset class

In [51]:
class CustomDataset(Dataset):
    def __init__(self, data, device):
        self.data = data
        self.device = device

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Assuming each item in the dataset is a tuple of (input, output)
        sample = self.data[index]
        input_array, output_array = sample[0], sample[1]

        # Convert to PyTorch tensors
        input_tensor = torch.tensor(input_array, dtype=dtype_to_use, device=self.device)
        output_tensor = torch.tensor(output_array, dtype=dtype_to_use, device=self.device)

        return input_tensor, output_tensor
    
class CustomDataModule(pl.LightningDataModule):
    def __init__(self, batch_size, num_workers, device):
        super().__init__()
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.device = device
        self.train_ds = None
        self.test_ds = None

    def prepare_data(self):
        pass

    #Arrays are transposed to make size 100 instead of 2
    def setup(self, stage):
        train_data = []
        test_data = []
        for subdir in os.listdir(train_dir):
            input = cv2.imread(os.path.join(train_dir, subdir, 'polar_plot.png'), cv2.IMREAD_GRAYSCALE) 
            output = pd.read_csv(os.path.join(train_dir, subdir, 'coords.csv')).values.transpose()
            train_data.append((input, output))
        for subdir in os.listdir(test_dir):
            input = cv2.imread(os.path.join(train_dir, subdir, 'polar_plot.png'), cv2.IMREAD_GRAYSCALE) 
            output = pd.read_csv(os.path.join(test_dir, subdir, 'coords.csv')).values.transpose()
            test_data.append((input, output))

        self.train_ds = CustomDataset(train_data, self.device)
        self.test_ds = CustomDataset(test_data, self.device)

    def train_dataloader(self):
        return DataLoader(
            self.train_ds,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_ds,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
        )

In [52]:
# Set device cuda for GPU if it's available otherwise run on the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
if device == torch.device("cuda"):
    print(f"Using: {torch.cuda.get_device_name(device)}")
else:
    print("Using: CPU")
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
lightning_model = NN(learning_rate=learning_rate, input_size=input_size, output_size=output_size).to(device)


dm = CustomDataModule(batch_size, num_workers, device)

Using: Quadro P600


# Train and plot loss over epochs, as well as average and maximum difference for every sample in the last epoch

In [53]:
#! Change this to your name so that each one has their results in a separate folder
tb_logger = TensorBoardLogger("logs", name="results_Marco")
trainer = pl.Trainer(max_epochs=num_epochs, logger=tb_logger)  # Adjust parameters as needed

# Train the model
trainer.fit(lightning_model, dm)
%tensorboard --logdir logs/results_Marco

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type      | Params
-------------------------------------
0 | layer1 | Conv2d    | 544   
1 | layer2 | Conv2d    | 32.8 K
2 | layer4 | Linear    | 13.9 M
3 | layer5 | Linear    | 40.2 K
4 | pool   | MaxPool2d | 0     
5 | relu   | ReLU      | 0     
-------------------------------------
14.0 M    Trainable params
0         Non-trainable params
14.0 M    Total params
56.052    Total estimated model params size (MB)
c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` 

Epoch 91:  18%|█▊        | 24/130 [00:07<00:33,  3.19it/s, v_num=13] 

c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
UsageError: Line magic function `%tensorboard` not found.


In [55]:
trainer = pl.Trainer(max_epochs=92, logger=tb_logger)  # Adjust parameters as needed

# Train the model
trainer.fit(lightning_model, dm, ckpt_path=r".\logs\results_Marco\version_10\checkpoints\epoch=90-step=11830.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at .\logs\results_Marco\version_10\checkpoints\epoch=90-step=11830.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type      | Params
-------------------------------------
0 | layer1 | Conv2d    | 544   
1 | layer2 | Conv2d    | 32.8 K
2 | layer4 | Linear    | 13.9 M
3 | layer5 | Linear    | 40.2 K
4 | pool   | MaxPool2d | 0     
5 | relu   | ReLU      | 0     
-------------------------------------
14.0 M    Trainable params
0         Non-trainable params
14.0 M    Total params
56.052    Total estimated model params size (MB)
Restored all states from the checkpoint at .\logs\results_Marco\version_10\checkpoints\epoch=90-step=11830.ckpt
c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarn

Epoch 91: 100%|██████████| 130/130 [00:41<00:00,  3.10it/s, v_num=13]

`Trainer.fit` stopped: `max_epochs=92` reached.


Epoch 91: 100%|██████████| 130/130 [00:44<00:00,  2.90it/s, v_num=13]


# Get results from testing:


In [56]:
trainer.test(lightning_model, dm)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

TypeError: must be real number, not NoneType